# make sure you dont use the internet. 
this notbook run for me 10 hours... stay tune for updates

In [28]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import h5py
from io import BytesIO

In [29]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms, v2
import torchvision.models as models

In [30]:
torch.manual_seed(1729)

In [31]:
device = ("cuda" if torch.cuda.is_available() 
          else "mps" if torch.backends.mps.is_available() 
          else "cpu")
torch.set_default_device(device)
print(f"device: {device}")

device: cuda


# IMPORTANT !!!    remember to update MODEL_PATH !!!
* upload as a model in the right top corner (in kaggle)
* copy its path and paste it in MODEl_PATH in IN_KAGGLE

In [32]:
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IN_COLAB = "COLAB_GPU" in os.environ

if IN_COLAB:
    from google.colab import userdata    
    # only the first time
    # os.environ["KAGGLE_KEY"] = "d0ebb5786a5a5439881827924cf0ccbd"
    # os.environ["KAGGLE_USERNAME"] = "yuda03979"
    # 
    # ! kaggle competitions download isic-2024-challenge
    # ! unzip isic-2024-challenge.zip
    MODEL_PATH = "?"
    IMG_DIR = "/content/test-image.hdf5"
    CSV_TEST_PATH = "/content/test-metadata.csv"
elif IN_KAGGLE:
    MODEL_PATH = "_model_.pth"
    IMG_DIR = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
    CSV_TEST_PATH = "/kaggle/input/isic-2024-challenge/test-metadata.csv"
else:
    MODEL_PATH = "_model_.pth"
    IMG_DIR = "/Users/yuda/Desktop/data_bases/isic-2024-challenge/test-image.hdf5"
    CSV_TEST_PATH = "/Users/yuda/PycharmProjects/my_isic2024/isic-2024-challenge/test-metadata.csv"

# create custom Dataset with reading hdf5

In [33]:
class ISICDataset(Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.file = h5py.File(IMG_DIR,'r')

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        binary_data = self.file[self.df.loc[idx, "isic_id"]][()]
        img = Image.open(BytesIO(binary_data)).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        return img


# choose/define your model (its layers and functions)

In [34]:
def load_model(model_name="mobilenet_v3_small", num_classes=1):

    if model_name == "mobilenet_v3_small":
        model = models.mobilenet_v3_small(weights=False)
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
    
    elif model_name == "efficientnet_v2_m":
        model = models.efficientnet_v2_m(weights=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(weights=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    elif model_name == "vgg16":
        model = models.vgg16(weights=False)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    
    elif model_name == "resnet50":
        model = models.resnet50(weights=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    elif model_name == "resnet18":
        model = models.resnet18(weights=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    elif model_name == "vit_b_16":
        model = models.vit_b_16(weights=False)
        in_features = model.heads.head.in_features
        model.heads.head = nn.Linear(in_features, num_classes)
    else:
        raise ValueError(f"Model {model_name} is not supported yet")
        
#     print(model)
    model = model.to(device)

    return model

In [35]:
# an example. you also can make a class or python-function
model = load_model() # internet cant be use. make shure that the model_name is your model


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# upload model _model_.pth file and update the model weights

In [36]:
model = model.to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

/tmp/ipykernel_36/4039808982.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [37]:
BATCH_SIZE = 128
IMG_SIZE = (224, 224)

In [38]:
ISIC_df = pd.read_csv(CSV_TEST_PATH)
ISIC_df = ISIC_df[["isic_id"]]
ISIC_df

,isic_id
0,ISIC_0015657
1,ISIC_0015729
2,ISIC_0015740


In [39]:
transform = transforms.Compose([transforms.Resize(IMG_SIZE),
                                v2.ToDtype(torch.float32, scale=True),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = [0.6298, 0.5126, 0.4097], std = [0.1386, 0.1308, 0.1202])])

In [40]:
test_dataset = ISICDataset(ISIC_df, transform)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) # cant be shuffled

# the predictions on the model

In [41]:
pred_arr = np.array([])
def test(dataloader, model):
    model.eval()
    global pred_arr
    with torch.no_grad():
        for batch, imgs in enumerate(dataloader):
            imgs = imgs.to(device)

            pred_labels = model(imgs)
            pred_labels = torch.sigmoid(pred_labels)
            
            pred_arr = np.concatenate((pred_arr, pred_labels.cpu().flatten().detach().numpy()), axis=None)
            
            # print(f"{batch} from {len(dataloader)}") # => to see the progress. you can also do if batch % 50 == 0 then print...
    return pred_arr


In [42]:
ISIC_df["target"] = test(test_dataloader, model)
ISIC_df.head()

,isic_id,target
0,ISIC_0015657,0.426453
1,ISIC_0015729,0.376777
2,ISIC_0015740,0.774313


In [43]:
ISIC_df.to_csv("submission.csv", index=False)